# Singlish Conversational AI Training

Training Qwen2.5-1.5B-Instruct for Singlish Q&A with reasoning preservation.
Includes fertility analysis and pre/post fine-tuning comparison.

# Installations
1. unsloth:
  * High-level library for fast, memory‑efficient fine‑tuning of LLMs (especially LoRA/QLoRA).
  * Wraps a lot of optimizations (quantization, custom kernels, config helpers) so you can train big models on limited GPUs.

2. bitsandbytes:
* Provides 8‑bit / 4‑bit quantization and optimized GPU ops.
* Used to load and fine‑tune very large models using less GPU memory (QLoRA-style training).

3. peft
* “Parameter‑Efficient Fine‑Tuning” library from Hugging Face.
* Implements LoRA, prefix-tuning, etc., so you can fine‑tune large models by training only a small number of extra parameters.


4. trl
* “Transformer Reinforcement Learning” library from Hugging Face.
* Used for RLHF and post‑training (PPO, DPO, ORPO, etc.) and supervised fine‑tuning pipelines on top of Transformers models.














In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers
!pip install --no-deps trl

In [ ]:
# Core imports
import torch
import time
import json
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template, train_on_responses_only
from trl import SFTTrainer, SFTConfig
import re

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: Could not find Trainer class in trl.trainer.bco_trainer. Found: ['BCOTrainer', '_BCOTrainer']


# Loading of Model and Lora Configs

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Instruct-2507",
    max_seq_length = 2048,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
)

==((====))==  Unsloth 2025.11.6: Fast Qwen3 patching. Transformers: 4.57.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.11.6 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


# Data Preparation

In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen3-instruct",
)

In [ ]:
dataset = load_dataset("csv", data_files="singlish_pairs.csv", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def convert_to_standard_format(example):
    return {
        "conversations": [
            {"role": "user", "content": example["Instruction"]},
            {"role": "assistant", "content": example["Output"]}
        ]
    }

# Apply the conversion
dataset = dataset.map(convert_to_standard_format)
dataset[100]

Map:   0%|          | 0/1284 [00:00<?, ? examples/s]

{'Instruction': 'Why always so tired?',
 'Output': 'Sleep not enough or sleep quality bad lor. Also check if you eat properly. If really cannot then see doctor lah.',
 'conversations': [{'content': 'Why always so tired?', 'role': 'user'},
  {'content': 'Sleep not enough or sleep quality bad lor. Also check if you eat properly. If really cannot then see doctor lah.',
   'role': 'assistant'}]}

In [ ]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)
dataset[100]['text']

Map:   0%|          | 0/1284 [00:00<?, ? examples/s]

'<|im_start|>user\nWhy always so tired?<|im_end|>\n<|im_start|>assistant\nSleep not enough or sleep quality bad lor. Also check if you eat properly. If really cannot then see doctor lah.<|im_end|>\n'

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 20,
        num_train_epochs = 2, # Set this for 1 full training run.
        # max_steps = 500,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/1284 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Map (num_proc=16):   0%|          | 0/1284 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<|im_start|>user\nWhy always so tired?<|im_end|>\n<|im_start|>assistant\nSleep not enough or sleep quality bad lor. Also check if you eat properly. If really cannot then see doctor lah.<|im_end|>\n'

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'             Sleep not enough or sleep quality bad lor. Also check if you eat properly. If really cannot then see doctor lah.<|im_end|>\n'

In [ ]:
# Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
3.816 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,284 | Num Epochs = 2 | Total steps = 322
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)


Step,Training Loss
10,4.868200
20,3.212300
30,2.492500
40,2.405100
50,2.199700
60,2.262500
70,2.255800
80,2.181200
90,2.112300
100,2.033700


In [ ]:
# Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

670.1816 seconds used for training.
11.17 minutes used for training.
Peak reserved memory = 4.338 GB.
Peak reserved memory for training = 0.522 GB.
Peak reserved memory % of max memory = 10.966 %.
Peak reserved memory for training % of max memory = 1.32 %.


# Test

In [ ]:
messages = [
    {"role" : "user", "content" : "HUH"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 100, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

What happened? Don't act blur lah.<|im_end|>


# Singlish Adapter

In [ ]:
# Save Singlish Adapter
model.save_pretrained("singlish_adapter")
tokenizer.save_pretrained("singlish_adapter")

print("Singlish adapter saved to singlish_adapter/")

Singlish adapter saved to singlish_adapter/


In [ ]:
# Clear VRAM first
# del model, trainer
torch.cuda.empty_cache()

# Load base model in FULL PRECISION (required for merging)
from unsloth import FastLanguageModel
from peft import PeftModel

base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen3-4B-Instruct-2507",
    max_seq_length=2048,
    load_in_4bit=False,  # MUST be False for merging
    load_in_8bit=False,
)

# Load Singlish adapter on top
model_with_adapter = PeftModel.from_pretrained(
    base_model,
    "singlish_adapter"
)

# Merge adapter weights into base model
print("Merging adapter into base model...")
merged_model = model_with_adapter.merge_and_unload()

# Save merged model (this is your new Singlish base ~8GB)
merged_model.save_pretrained("qwen3-4b-singlish-base")
tokenizer.save_pretrained("qwen3-4b-singlish-base")

print("Merged model saved to qwen3-4b-singlish-base/")

==((====))==  Unsloth 2025.11.6: Fast Qwen3 patching. Transformers: 4.57.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Merging adapter into base model...
Merged model saved to qwen3-4b-singlish-base/


In [ ]:
from google.colab import userdata
from huggingface_hub import HfApi, login

login(token=userdata.get('HF_TOKEN'))
api = HfApi()
repo_id = "yuhueng/qwen3-4b-singlish-base-v3"

api.create_repo(repo_id=repo_id, private=True, exist_ok=True)

# Upload the entire folder
api.upload_folder(
    folder_path="qwen3-4b-singlish-base",
    repo_id=repo_id,
    repo_type="model"
)


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...glish-base/tokenizer.json:   0%|          | 28.3kB / 11.4MB            

  ...0002-of-00002.safetensors:   0%|          |  603kB / 3.08GB            

  ...0001-of-00002.safetensors:   1%|1         | 50.3MB / 4.97GB            

CommitInfo(commit_url='https://huggingface.co/yuhueng/qwen3-4b-singlish-base-v3/commit/aae49717624128fa247be7260b43f0238bf5c90c', commit_message='Upload folder using huggingface_hub', commit_description='', oid='aae49717624128fa247be7260b43f0238bf5c90c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yuhueng/qwen3-4b-singlish-base-v3', endpoint='https://huggingface.co', repo_type='model', repo_id='yuhueng/qwen3-4b-singlish-base-v3'), pr_revision=None, pr_num=None)